# Alhazen Agent Gradio  / Chatbot.

> Building a simple Gradio application to allow the user to chat to Alhazen. 

In [ ]:
#| default_exp apps.gradio_chat

In [ ]:
#| export

from alhazen.core import MODEL_TYPE, PromptTemplateRegistry, load_alhazen_tool_environment, get_langchain_chatmodel
from alhazen.agent import AlhazenAgent

from alhazen.schema_sqla import *
from alhazen.tools.basic import IntrospectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import * 
from alhazen.toolkit import AlhazenToolkit
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database

import click

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, AgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.output_parsers.json import parse_json_markdown
from langchain.pydantic_v1 import BaseModel
from langchain.tools.render import render_text_description, render_text_description_and_args
from langchain.agents.output_parsers.json import JSONAgentOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool


from langchain.prompts import ChatPromptTemplate


import gradio as gr
import os
import pandas as pd
from pathlib import Path
import re
import requests
import sys
from typing import Dict, List, Optional, Sequence, Union, Any
from uuid import UUID

/Users/gully.burns/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export 

class AlhazenAgentChatBot(AlhazenAgent):
    '''Provide access to an Alhazen agent via a simple chat interface with Gradio.'''

    # We need to add these input/output schemas because the current AgentExecutor
    # is lacking in schemas.
    class ChatRequest(BaseModel):
        question: str
        chat_history: Optional[List[Dict[str, str]]]

    class Input(BaseModel):
        input: str

    class Output(BaseModel):
        output: str

    def run_gradio(self):

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)
            
        def clear_chat(history):
            return []

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            response = self.agent_executor.invoke({"input": prompt}, config={'callbacks': [ConsoleCallbackHandler()]})
            print('RESPONSE: %s'%(str(response)))
            history[-1][1] = str(response.get('output'))
            print('WHOLE HISTORY: %s'%(history))
            return history

        with gr.Blocks() as demo:
            with gr.Tab("Chat"):
                chatbot = gr.Chatbot(
                    [],
                    elem_id="chatbot",
                    bubble_full_width=False,
                    height="100%",
                    #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
                )
                with gr.Row():
                    txt = gr.Textbox(
                        scale=4,
                        show_label=False,
                        placeholder="Enter text and press enter.",
                        container=False,
                    )
                    clear_btn = gr.Button("Clear")
            with gr.Tab("Dashboard"):
                with gr.Row():
                    doc_text = gr.HTML(label="File Contents")
            
            txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
            txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)                
            clear_btn.click(clear_chat, [], [chatbot], queue=False)
                
        demo.queue()
        demo.launch()


In [ ]:
#| export
@click.command()
@click.option('--loc', default=None, help='Where on disk to put local files')
@click.option('--db_name', prompt=None, help='The name of the database being used.')
def alhazen_chatbot(loc, db_name):
    """Runs the Alhazen Chatbot."""
    if loc:
        os.environ['LOCAL_FILE_PATH'] = loc
    if db_name:
        os.environ['ALHAZEN_DB_NAME'] = db_name
    cb = AlhazenAgentGradioChatBot()
    cb.run_gradio()


NameError: name 'click' is not defined

In [ ]:
#| export
#|eval: false
if __name__ == '__main__':
    alhazen_chatbot()
    

Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: --f


AttributeError: 'tuple' object has no attribute 'tb_frame'